In [1]:
import pandas as pd 
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
#ccba
ccba = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_ccba_full_hashed.csv')
#cdtx
cdtx = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_cdtx0001_full_hashed.csv')
#custinfo
cdtx2 = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_custinfo_full_hashed.csv')
#cdtx
dp = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_dp_full_hashed.csv')
#remit
remit = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_train_x_remit1_full_hashed.csv')
#alert_time
alert_time = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\public_x_alert_date.csv')
#y
y = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\train_y_answer.csv')
#案件名單
doc = pd.read_csv(r'E:\Desktop\玉山\訓練資料集_first\預測的案件名單及提交檔案範例.csv')

In [3]:
cdtx['country_org'] = cdtx['country']
cdtx['cur_type_org'] = cdtx['cur_type']

In [4]:
cdtx['country'] = (cdtx['country'] == 130).astype(int)
cdtx['cur_type'] = (cdtx['cur_type'] == 47).astype(int)

In [5]:
cdtx['country'] = cdtx['country'].apply(lambda x: 'TW' if x == 1 else 'FN')
cdtx['cur_type'] = cdtx['cur_type'].apply(lambda x: 'TWD' if x == 1 else 'FND')

In [6]:
doc_merged = cdtx2.merge(doc, on='alert_key', how='right').merge(alert_time, on='alert_key', how='left')
doc_merged = doc_merged[~doc_merged['cust_id'].isnull()]
max_doc = doc_merged.groupby('cust_id')['date'].max().reset_index()
max_doc.rename(columns={'date':'max_date'},inplace=True)
doc_merged = doc_merged.merge(max_doc, on='cust_id', how='left')
prev_list = doc_merged[doc_merged['max_date']>doc_merged['date']]['alert_key'].tolist()

In [7]:
alert_cdtx = cdtx[cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx = cdtx[~cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(alert_time['alert_key'].tolist())]['cust_id'].tolist())]
train_cdtx['y'] = 0

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
train_cdtx[train_cdtx.cust_id == '36c96f886054b13855e0531cfc53562de5acdada08fb75c24e6d957c0d8af1a9']

,cust_id,date,country,cur_type,amt,country_org,cur_type_org,y
285147,36c96f886054b13855e0531cfc53562de5acdada08fb75...,33,TW,TWD,12926.0,130,47,0
285148,36c96f886054b13855e0531cfc53562de5acdada08fb75...,41,TW,TWD,667.0,130,47,0
285149,36c96f886054b13855e0531cfc53562de5acdada08fb75...,69,TW,TWD,924.0,130,47,0
285150,36c96f886054b13855e0531cfc53562de5acdada08fb75...,183,FN,TWD,1319.0,119,47,0
285151,36c96f886054b13855e0531cfc53562de5acdada08fb75...,310,TW,TWD,1641.0,130,47,0
285152,36c96f886054b13855e0531cfc53562de5acdada08fb75...,310,TW,TWD,1604.0,130,47,0
285153,36c96f886054b13855e0531cfc53562de5acdada08fb75...,318,TW,TWD,4733.0,130,47,0
285154,36c96f886054b13855e0531cfc53562de5acdada08fb75...,333,TW,TWD,1006.0,130,47,0
285155,36c96f886054b13855e0531cfc53562de5acdada08fb75...,333,TW,TWD,923.0,130,47,0
285156,36c96f886054b13855e0531cfc53562de5acdada08fb75...,378,TW,TWD,2033.0,130,47,0


In [9]:
sar_idx = train_cdtx[train_cdtx['cust_id'].isin(cdtx2[cdtx2['alert_key'].isin(y[y['sar_flag']==1]['alert_key'].tolist())]['cust_id'].tolist())].index
train_cdtx.loc[sar_idx, 'y'] = 1

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
alert_time = cdtx2.merge(alert_time)[['alert_key', 'cust_id','date']]
alert_cdtx = alert_cdtx.merge(alert_time)

In [11]:
#Data Cleansing
#SAR戶僅留前X天
def sar_pred_d(x, day):
  prev_d = x.groupby('cust_id')['date'].max() - day
  prev_d = prev_d.reset_index()
  prev_d.rename(columns={'date':'prev_d'}, inplace=True)
  x = x.merge(prev_d, on='cust_id', how='left')
  x.drop(x[x['date']<x['prev_d']].index,inplace=True)
  x.pop('prev_d')
  return x

train_cdtx = sar_pred_d(train_cdtx, 30)
alert_cdtx = sar_pred_d(alert_cdtx, 30)


In [12]:
train_cdtx = pd.merge(train_cdtx, cdtx2)
alert_cdtx = pd.merge(alert_cdtx, cdtx2)

In [13]:
class renamer():
    def __init__(self):
        self.d = dict()
    def __call__(self, x):
        if x not in self.d:
            self.d[x] =  0
            return x
        else:
            self.d[x] += 1
            return "%s_%d" % (x, self.d[x])

In [15]:
def total_amt_rolling_3(data):
    total_amt_rolling_3 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(3, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_3.columns = ['cust_id', 'alert_key', 'total_amt_rolling_3']
    on_day_total_amt = train_cdtx.groupby(['cust_id', 'date', 'alert_key'])['amt'].sum().reset_index()
    on_day_total_amt.columns = ['cust_id', 'date', 'alert_key', 'on_day_total_amt']
    data = pd.concat([data, total_amt_rolling_3], axis=1)
    data = data.rename(columns=renamer())  
    data = pd.merge(data, on_day_total_amt, how='left') 
    data['total_amt_rolling_3'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_3']) else x['total_amt_rolling_3'], axis=1)
    return data

train_cdtx = total_amt_rolling_3(train_cdtx)
alert_cdtx = total_amt_rolling_3(alert_cdtx)

In [16]:
def total_amt_rolling_7(data):
    total_amt_rolling_7 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(7, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_7.columns = ['cust_id', 'alert_key', 'total_amt_rolling_7']
    data = pd.concat([data, total_amt_rolling_7], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_7'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_7']) else x['total_amt_rolling_7'], axis=1)
    return data
train_cdtx = total_amt_rolling_7(train_cdtx)
alert_cdtx = total_amt_rolling_7(alert_cdtx)

In [17]:
def total_amt_rolling_14(data):
    total_amt_rolling_14 = (data[data.cur_type == 'TWD'].groupby(['cust_id', 'alert_key'])
                   .apply(lambda g: g.rolling(14, on='date')['amt'].sum())).reset_index().drop(['level_2'], axis=1)
    total_amt_rolling_14.columns = ['cust_id', 'alert_key', 'total_amt_rolling_14']
    data = pd.concat([data, total_amt_rolling_14], axis=1)
    data = data.rename(columns=renamer())
    data['total_amt_rolling_14'] = data.apply(lambda x: x['on_day_total_amt'] if np.isnan(x['total_amt_rolling_14']) else x['total_amt_rolling_14'], axis=1)
    return data
train_cdtx = total_amt_rolling_14(train_cdtx)
alert_cdtx = total_amt_rolling_14(alert_cdtx)

In [18]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].count().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    session_amt_diff['TWD'] / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [19]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [20]:
def amt_diff_func(data):
  session_amt_diff = data.groupby(['cust_id','country'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='country', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['country_amt_diff_ratio'] = \
    abs(session_amt_diff['TW'] - session_amt_diff['FN']) / abs(session_amt_diff['TW'] + session_amt_diff['FN'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','country_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')
  

  session_amt_diff = data.groupby(['cust_id','cur_type'])['amt'].sum().reset_index()
  session_amt_diff = pd.pivot_table(session_amt_diff, index='cust_id', columns='cur_type', values='amt')
  session_amt_diff.fillna(1, inplace=True)
  session_amt_diff['cur_amt_diff_ratio'] = \
    abs(session_amt_diff['TWD'] - session_amt_diff['FND']) / abs(session_amt_diff['TWD'] + session_amt_diff['FND'])
  session_amt_diff = session_amt_diff.reset_index()[['cust_id','cur_amt_diff_ratio']]
  data = data.merge(session_amt_diff, on='cust_id', how='left')

  return data

In [21]:
train_cdtx = amt_diff_func(train_cdtx)
alert_cdtx = amt_diff_func(alert_cdtx)

In [22]:
train_cdtx = train_cdtx.rename(columns=renamer())
alert_cdtx = alert_cdtx.rename(columns=renamer())

In [23]:
train_cdtx['mix_country_cur_type'] = train_cdtx['country_org']*100 + train_cdtx['cur_type_org']
train_cdtx['mix_country_cur_type'] = train_cdtx['country_org']*100 + train_cdtx['cur_type_org']
alert_cdtx['mix_country_cur_type'] = alert_cdtx['country_org']*100 + alert_cdtx['cur_type_org']
alert_cdtx['mix_country_cur_type'] = alert_cdtx['country_org']*100 + alert_cdtx['cur_type_org']

In [24]:
cdtx

,cust_id,date,country,cur_type,amt,country_org,cur_type_org
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,1,TW,TWD,2312.0,130,47
1,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,10,TW,TWD,808.0,130,47
2,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,10,TW,TWD,7099.0,130,47
3,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,13,TW,TWD,306.0,130,47
4,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,15,TW,TWD,255.0,130,47
...,...,...,...,...,...,...,...
1043009,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,130,47
1043010,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,130,47
1043011,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,130,47
1043012,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,130,47


In [25]:
d = cdtx.groupby(['cust_id', 'date'])['amt'].count().reset_index()

In [26]:
d['1d_cnt'] = d['amt'].apply(lambda x: 1 if x <= 50 else 2 if x<=200 else 3)
d.drop(['amt'], axis=1, inplace=True)

In [27]:
train_cdtx = train_cdtx.merge(d, how='left')
alert_cdtx = alert_cdtx.merge(d, how='left')

In [28]:
bins = [-np.inf, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 10000, np.inf]
labels = [x for x in range(len(bins)-1)]

train_cdtx['amt_group'] = pd.cut(train_cdtx.amt, bins=bins, labels=labels)
alert_cdtx['amt_group'] = pd.cut(train_cdtx.amt, bins=bins, labels=labels)

In [29]:
train_cdtx['country'] = train_cdtx['country'].astype('category')
train_cdtx['cur_type'] = train_cdtx['cur_type'].astype('category')

alert_cdtx['country'] = alert_cdtx['country'].astype('category')
alert_cdtx['cur_type'] = alert_cdtx['cur_type'].astype('category')

train_cdtx['risk_rank'] = train_cdtx['risk_rank'].astype('category')
train_cdtx['occupation_code'] = train_cdtx['occupation_code'].astype('category')
train_cdtx['AGE'] = train_cdtx['AGE'].astype('category')
alert_cdtx['risk_rank'] = alert_cdtx['risk_rank'].astype('category')
alert_cdtx['occupation_code'] = alert_cdtx['occupation_code'].astype('category')
alert_cdtx['AGE'] = alert_cdtx['AGE'].astype('category')

In [30]:
train_cdtx['mix_country_cur_type'] = train_cdtx['mix_country_cur_type'].astype('category')
alert_cdtx['mix_country_cur_type'] = alert_cdtx['mix_country_cur_type'].astype('category')

In [31]:
cat_cols = ['cust_id', 'date', 'risk_rank', 'occupation_code', 'AGE']
train_cdtx = train_cdtx.sort_values(['cust_id', 'date'])
train_cdtx['risk_rank'] = train_cdtx['risk_rank'].ffill()
train_cdtx['occupation_code'] = train_cdtx['occupation_code'].ffill()
train_cdtx['AGE'] = train_cdtx['AGE'].ffill()

In [32]:
train_cdtx

,cust_id,date,country,cur_type,amt,country_org,cur_type_org,y,alert_key,risk_rank,...,cust_id_1_1,alert_key_1_1,total_amt_rolling_14,country_amt_diff_ratio_x,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,mix_country_cur_type,1d_cnt,amt_group
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,TW,TWD,237.0,130,47,0,195913,1,...,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,237.0,0.333333,0.998462,0.996923,0.996923,13047,1,2
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,TW,TWD,412.0,130,47,0,195913,1,...,0015cc01f553512007705edf855baa16f1b57a16fb107e...,195913.0,412.0,0.333333,0.998462,0.996923,0.996923,13047,1,4
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,269235,3,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991,13047,1,5
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,270967,3,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991,13047,1,5
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,271556,3,...,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,269235.0,22874.0,0.953488,0.999996,0.999991,0.999991,13047,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,130,47,0,303894,1,...,NaN,NaN,229.0,0.916667,0.999975,0.976025,0.999951,13047,1,2
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,130,47,0,303894,1,...,NaN,NaN,1112.0,0.916667,0.999975,0.976025,0.999951,13047,1,10
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,130,47,0,303894,1,...,NaN,NaN,59.0,0.916667,0.999975,0.976025,0.999951,13047,1,0
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,130,47,0,303894,1,...,NaN,NaN,156.0,0.916667,0.999975,0.976025,0.999951,13047,1,1


In [33]:
train_cdtx['ratio_total_aum_7'] = train_cdtx.total_amt_rolling_7 /train_cdtx.total_asset
train_cdtx['ratio_total_aum_1'] = train_cdtx.on_day_total_amt  /train_cdtx.total_asset 
train_cdtx['ratio_total_aum_14'] = train_cdtx.total_amt_rolling_14/train_cdtx.total_asset
train_cdtx['ratio_total_aum_3'] = train_cdtx.total_amt_rolling_3  /train_cdtx.total_asset
alert_cdtx['ratio_total_aum_7'] = alert_cdtx.total_amt_rolling_7 /alert_cdtx.total_asset
alert_cdtx['ratio_total_aum_1'] = alert_cdtx.on_day_total_amt  /alert_cdtx.total_asset 
alert_cdtx['ratio_total_aum_14'] = alert_cdtx.total_amt_rolling_14/alert_cdtx.total_asset
alert_cdtx['ratio_total_aum_3'] = alert_cdtx.total_amt_rolling_3  /alert_cdtx.total_asset

In [34]:
train_cdtx

,cust_id,date,country,cur_type,amt,country_org,cur_type_org,y,alert_key,risk_rank,...,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,mix_country_cur_type,1d_cnt,amt_group,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,TW,TWD,237.0,130,47,0,195913,1,...,0.998462,0.996923,0.996923,13047,1,2,0.007235,0.007235,0.007235,0.007235
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,TW,TWD,412.0,130,47,0,195913,1,...,0.998462,0.996923,0.996923,13047,1,4,0.012577,0.012577,0.012577,0.012577
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,269235,3,...,0.999996,0.999991,0.999991,13047,1,5,0.036320,0.036320,0.036320,0.036320
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,270967,3,...,0.999996,0.999991,0.999991,13047,1,5,0.036320,0.036320,0.036320,0.036320
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,TW,TWD,508.0,130,47,0,271556,3,...,0.999996,0.999991,0.999991,13047,1,5,0.036320,0.036320,0.036320,0.036512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,TW,TWD,229.0,130,47,0,303894,1,...,0.999975,0.976025,0.999951,13047,1,2,0.000500,0.000500,0.000500,0.000500
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,TW,TWD,1112.0,130,47,0,303894,1,...,0.999975,0.976025,0.999951,13047,1,10,0.002426,0.002426,0.002426,0.002426
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,TW,TWD,59.0,130,47,0,303894,1,...,0.999975,0.976025,0.999951,13047,1,0,0.000129,0.000129,0.000129,0.000129
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,TW,TWD,156.0,130,47,0,303894,1,...,0.999975,0.976025,0.999951,13047,1,1,0.000340,0.000340,0.000340,0.000340


In [35]:
alert_cdtx.replace([np.inf, -np.inf], 1, inplace=True)

In [36]:
train_cdtx.columns

Index(['cust_id', 'date', 'country', 'cur_type', 'amt', 'country_org',
       'cur_type_org', 'y', 'alert_key', 'risk_rank', 'occupation_code',
       'total_asset', 'AGE', 'cust_id_1', 'alert_key_1', 'total_amt_rolling_3',
       'on_day_total_amt', 'cust_id_1_1', 'alert_key_1_1',
       'total_amt_rolling_7', 'cust_id_1_1', 'alert_key_1_1',
       'total_amt_rolling_14', 'country_amt_diff_ratio_x',
       'cur_amt_diff_ratio_x', 'country_amt_diff_ratio_y',
       'cur_amt_diff_ratio_y', 'mix_country_cur_type', '1d_cnt', 'amt_group',
       'ratio_total_aum_7', 'ratio_total_aum_1', 'ratio_total_aum_14',
       'ratio_total_aum_3'],
      dtype='object')

In [37]:
cat_cols = train_cdtx.select_dtypes(include=["category"]).columns

In [38]:
train_cdtx.loc[:, ~train_cdtx.columns.isin(cat_cols)].fillna(0,inplace=True)
alert_cdtx.loc[:, ~alert_cdtx.columns.isin(cat_cols)].fillna(0,inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [39]:
train_col = ['cust_id', 'date', 'alert_key',#, 'amt', 'country', 'cur_type'
            'mix_country_cur_type', '1d_cnt', 'amt_group',
       #,'risk_rank', 'occupation_code', 'total_asset', 'AGE'
       #     , 'lupay', 'cycam',
       #'usgam'#, 'clamt'
       #      , 'csamt', 'inamt', 'cucsm', 'cucah',
       'total_amt_rolling_3', 'on_day_total_amt',
       'total_amt_rolling_7', 'total_amt_rolling_14', 'country_amt_diff_ratio_x',
       'cur_amt_diff_ratio_x', 'country_amt_diff_ratio_y',
       'cur_amt_diff_ratio_y', 'ratio_total_aum_7', 'ratio_total_aum_1',
       'ratio_total_aum_14', 'ratio_total_aum_3',
        'y']
x_col = train_col[3:-1]
train_data = train_cdtx[train_col]
train_data

,cust_id,date,alert_key,mix_country_cur_type,1d_cnt,amt_group,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,country_amt_diff_ratio_x,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3,y
0,0015cc01f553512007705edf855baa16f1b57a16fb107e...,372,195913,13047,1,2,237.0,237.0,237.0,237.0,0.333333,0.998462,0.996923,0.996923,0.007235,0.007235,0.007235,0.007235,0
1,0015cc01f553512007705edf855baa16f1b57a16fb107e...,386,195913,13047,1,4,412.0,412.0,412.0,412.0,0.333333,0.998462,0.996923,0.996923,0.012577,0.012577,0.012577,0.012577,0
2,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,269235,13047,1,5,22874.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320,0
3,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,270967,13047,1,5,22874.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320,0
4,002d6bc6382e759c9a5b6bb2c8dda9add2f38cd87e10ca...,364,271556,13047,1,5,22995.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036512,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,359,303894,13047,1,2,229.0,229.0,229.0,229.0,0.916667,0.999975,0.976025,0.999951,0.000500,0.000500,0.000500,0.000500,0
378485,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,360,303894,13047,1,10,1112.0,1112.0,1112.0,1112.0,0.916667,0.999975,0.976025,0.999951,0.002426,0.002426,0.002426,0.002426,0
378486,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,361,303894,13047,1,0,59.0,59.0,59.0,59.0,0.916667,0.999975,0.976025,0.999951,0.000129,0.000129,0.000129,0.000129,0
378487,ffeb846f2294ec4397f800736dc1e14eb754da5425398d...,364,303894,13047,1,1,156.0,156.0,156.0,156.0,0.916667,0.999975,0.976025,0.999951,0.000340,0.000340,0.000340,0.000340,0


In [40]:
train_data = train_data.drop_duplicates()

In [41]:
Y = train_data['y']
X = train_data[x_col]

In [42]:
X

,mix_country_cur_type,1d_cnt,amt_group,total_amt_rolling_3,on_day_total_amt,total_amt_rolling_7,total_amt_rolling_14,country_amt_diff_ratio_x,cur_amt_diff_ratio_x,country_amt_diff_ratio_y,cur_amt_diff_ratio_y,ratio_total_aum_7,ratio_total_aum_1,ratio_total_aum_14,ratio_total_aum_3
0,13047,1,2,237.0,237.0,237.0,237.0,0.333333,0.998462,0.996923,0.996923,0.007235,0.007235,0.007235,0.007235
1,13047,1,4,412.0,412.0,412.0,412.0,0.333333,0.998462,0.996923,0.996923,0.012577,0.012577,0.012577,0.012577
2,13047,1,5,22874.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320
3,13047,1,5,22874.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036320
4,13047,1,5,22995.0,22874.0,22874.0,22874.0,0.953488,0.999996,0.999991,0.999991,0.036320,0.036320,0.036320,0.036512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378484,13047,1,2,229.0,229.0,229.0,229.0,0.916667,0.999975,0.976025,0.999951,0.000500,0.000500,0.000500,0.000500
378485,13047,1,10,1112.0,1112.0,1112.0,1112.0,0.916667,0.999975,0.976025,0.999951,0.002426,0.002426,0.002426,0.002426
378486,13047,1,0,59.0,59.0,59.0,59.0,0.916667,0.999975,0.976025,0.999951,0.000129,0.000129,0.000129,0.000129
378487,13047,1,1,156.0,156.0,156.0,156.0,0.916667,0.999975,0.976025,0.999951,0.000340,0.000340,0.000340,0.000340


In [43]:
X.replace([np.inf, -np.inf], 1, inplace=True)

C:\Users\rocker\anaconda3\lib\site-packages\pandas\core\frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [44]:
#model training 1
#逐筆交易處理
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
model = XGBClassifier(
        tree_method='gpu_hist', 
        gpu_id=0,
        enable_categorical=True,
        use_label_encoder=False,
        predictor= "gpu_predictor")
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
#previous 30days label of 1
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 73.76%
#0.9921875
#0.5869618097786427

F1 score: 99.81%
0.9999452354874042
0.9963439921423115


In [45]:
pd.DataFrame({'columns':list(X.columns),'score':model.feature_importances_})

,columns,score
0,mix_country_cur_type,0.162137
1,1d_cnt,0.022258
2,amt_group,0.004166
3,total_amt_rolling_3,0.015080
4,on_day_total_amt,0.003646
5,total_amt_rolling_7,0.003036
6,total_amt_rolling_14,0.002715
7,country_amt_diff_ratio_x,0.104153
8,cur_amt_diff_ratio_x,0.524052
9,country_amt_diff_ratio_y,0.065354


In [46]:
#彙整model training 1偵測結果
#predict_proba以0.05機率區間為一個欄位判斷分佈
def mapping(proba):
    if proba <= 0.05:
        return 1
    elif 0.05 < proba <= 0.1:
        return 2
    elif 0.1 < proba <= 0.15:
        return 3
    elif 0.15 < proba <= 0.2:
        return 4
    elif 0.2 < proba <= 0.25:
        return 5
    elif 0.25 < proba <= 0.3:
        return 6
    elif 0.3 < proba <= 0.35:
        return 7
    elif 0.35 < proba <= 0.4:
        return 8
    elif 0.4 < proba <= 0.45:
        return 9
    elif 0.45 < proba <= 0.5:
        return 10
    elif 0.5 < proba <= 0.55:
        return 11
    elif 0.55 < proba <= 0.6:
        return 12
    elif 0.6 < proba <= 0.65:
        return 13
    elif 0.65 < proba <= 0.7:
        return 14
    elif 0.7 < proba <= 0.75:
        return 15
    elif 0.75 < proba <= 0.8:
        return 16
    elif 0.8 < proba <= 0.85:
        return 17
    elif 0.85 < proba <= 0.9:
        return 18
    elif 0.9 < proba <= 0.95:
        return 19
    elif 0.95 < proba <= 1:
        return 20

In [47]:
train_data['proba'] = model.predict_proba(X)[:,1]

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
result = train_data[['cust_id','proba','y']]
result['level'] = result["proba"].map(mapping)
result = result[['cust_id', 'level', 'y']]
result = result.groupby(['cust_id','level']).count().reset_index().pivot_table(index='cust_id', columns='level', values='y')
result.fillna(0, inplace=True)
result = result.div(result.sum(axis=1), axis=0)
result = result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(result.columns[1:])):
  result[col] = 0.0
result = result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
result = result.merge(train_data[['cust_id','y']].drop_duplicates(), on='cust_id', how='left')
result.pop('cust_id')

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
result.to_csv('cdtx.csv', index=False)

In [195]:
#model training 2
#歸戶判斷是否報SAR
result_col = list(result.columns)
result_col.remove('y')
test_size = 0.2
seed = 42
X_train, X_test, y_train, y_test = train_test_split(result[result_col], result['y'], test_size=test_size, random_state=seed)
model_2nd = XGBClassifier()
model_2nd.fit(X_train, y_train)
y_pred = model_2nd.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1 * 100.0))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
#F1 score: 36.36%
#0.7142857142857143
#0.24390243902439024

F1 score: 83.33%
0.8333333333333334
0.8333333333333334


In [196]:
#Submission
alert_x = alert_cdtx[x_col]
alert_data = alert_cdtx
alert_data['proba'] = model.predict_proba(alert_x)[:,1]
alert_result = alert_data[['cust_id','proba']]
alert_result['level'] = alert_result["proba"].map(mapping)
alert_result = alert_result[['cust_id', 'level']]
alert_result['cnt'] = 1
alert_result = alert_result.groupby(['cust_id','level'])['cnt'].count().reset_index().pivot_table(index='cust_id', columns='level', values='cnt')
alert_result.fillna(0, inplace=True)
alert_result = alert_result.div(alert_result.sum(axis=1), axis=0)
alert_result = alert_result.reset_index()
for col in list(set([num for num in range(1,21,1)]) -  set(alert_result.columns[1:])):
  alert_result[col] = 0.0
alert_result = alert_result[['cust_id', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
#alert_result.pop('cust_id')
alert_pred = model_2nd.predict_proba(alert_result[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]])
# evaluate predictions
final = alert_result[['cust_id']]
final['probability'] = alert_pred[:,1]
final

C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\rocker\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


level,cust_id,probability
0,0174dff6a8a2038f9966d61d8450e19e3bd751e4051c2a...,0.000849
1,01c705b457297805c1c08137cb3adb164b8f393c265db2...,0.000849
2,02e2cb9bb3a904b4e44b7e40f54d536ca5941db440ba00...,0.000849
3,035e2ebcd5c23f4dd0f1fc81a1fca8afd3121f7431bb58...,0.000849
4,03d4cbfcb065501bebb2b75ea4df2b779df0192a183af3...,0.000849
...,...,...
419,fec476c1d12e20806dc384748798a07bfd972ed0626890...,0.000849
420,fee6ee67b6db61c5a7788074e412b0c0ac77b3762b3b98...,0.000849
421,ff483c74ef2977bfcf04be0a79363af72d1812aac9ce1f...,0.000849
422,ff538fed253fce00e25a7c1557ee7728a5d8dc9c1bbfc1...,0.000849


In [149]:
alert_result.to_csv('cdtx_predict.csv', index=False)

In [197]:
final = alert_time.merge(cdtx2[['alert_key', 'cust_id']].merge(final, on='cust_id'), on='alert_key', how='left')#[['alert_key', 'probability']]

In [198]:
final.fillna(0, inplace=True)
final

,alert_key,date,cust_id,probability
0,352342,365,7629f010080267f680a4ca9c6cac71c4d4d86933dcc7ed...,0.000849
1,352866,365,0,0.000000
2,352696,365,c6def618ad861703c025be4f41bdf7569310228ae93aef...,0.000849
3,352330,365,0,0.000000
4,352683,365,0,0.000000
...,...,...,...,...
1840,364472,393,bdda3ef79c5f052fd281c9f28f50a25977444eceaf9751...,0.000849
1841,364788,393,0,0.000000
1842,364673,393,0,0.000000
1843,364626,393,0,0.000000


In [199]:
doc = doc[['alert_key']]
doc

,alert_key
0,357307
1,376329
2,373644
3,357668
4,354443
...,...
3845,364485
3846,363155
3847,368710
3848,358067


In [200]:
final = doc.merge(final, on='alert_key', how='left')
final.fillna(0,inplace=True)
final

,alert_key,date,cust_id,probability
0,357307,370.0,1d69b4daf9c5c8b8d68b1776193b6e80bf2e84c999d038...,0.000849
1,376329,0.0,0,0.000000
2,373644,0.0,0,0.000000
3,357668,370.0,8b51184740375f7ccdd68484aeeaca44c5892818eda908...,0.000849
4,354443,372.0,c6def618ad861703c025be4f41bdf7569310228ae93aef...,0.000849
...,...,...,...,...
3845,364485,393.0,f41c0860cf0cb5e7b72b0ced16ab427a06fc0ad3fc0f71...,0.000849
3846,363155,392.0,823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a...,0.000849
3847,368710,0.0,0,0.000000
3848,358067,382.0,0,0.000000


In [201]:
final.loc[final[final['alert_key'].isin(prev_list)].index,'probability']=0

In [202]:
final

,alert_key,date,cust_id,probability
0,357307,370.0,1d69b4daf9c5c8b8d68b1776193b6e80bf2e84c999d038...,0.000849
1,376329,0.0,0,0.000000
2,373644,0.0,0,0.000000
3,357668,370.0,8b51184740375f7ccdd68484aeeaca44c5892818eda908...,0.000849
4,354443,372.0,c6def618ad861703c025be4f41bdf7569310228ae93aef...,0.000000
...,...,...,...,...
3845,364485,393.0,f41c0860cf0cb5e7b72b0ced16ab427a06fc0ad3fc0f71...,0.000849
3846,363155,392.0,823fc5ce48cd827628ce0d1c574e6c3582cb772cf6173a...,0.000000
3847,368710,0.0,0,0.000000
3848,358067,382.0,0,0.000000


In [203]:
final = final[['alert_key', 'probability']]

In [144]:
final.to_csv('cdtx_aml.csv')

In [204]:
final[final['probability']>0]

,alert_key,probability
0,357307,0.000849
3,357668,0.000849
5,361128,0.000849
10,362463,0.000849
29,357471,0.000849
...,...,...
3803,361028,0.000849
3806,354423,0.000849
3841,353051,0.000849
3844,360534,0.000849
